In [1]:
import sys
sys.path.append("reinvent/")

In [2]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import pandas as pd
import numpy as np 
import scipy.stats as sps
from tqdm import tqdm, trange
from models import dataset
from models.model import Model
from utils.smiles import standardize_smiles
import utils.general

if utils.general.set_default_device_cuda():
    print("GPU is used")
else:
    print("Using CPU")
    

GPU is used


RDKit WARNING: [14:31:56] Enabling RDKit 2019.09.1 jupyter extensions


In [4]:
data = pd.read_csv("data/pubchem_assay_compounds_processed.tsv", sep='\t')

In [5]:
from glob import glob

calculation = len(data)*200
for i in tqdm(range(0,201)):
    if i == 0:
        agent = f"reinvent/data/augmented.prior"
    elif i == 200: 
        agent = f"output/focused.agent"
    else:
        agent = f"output/focused.agent.{i}"
    focused_agent = Model.load_from_file(agent)
    utils.general.set_seed(1234567890)
    def get_nlls():
        generator, length = dataset.calculate_nlls_from_model(focused_agent, data["rdkit_smiles"].to_list(), batch_size=250)
        nlls = []
        for nll in generator:
            nlls.append(nll)
        return  np.hstack(nlls)

    data[f"NLL_{i}"] = get_nlls()
    if i % 50 == 0:
        data.to_csv(f"output/promiscuous_processed_{i}.csv", index=False)
data.to_csv(f"output/promiscuous_processed.csv", index=False)


100%|██████████| 201/201 [1:21:06<00:00, 24.21s/it]
